# Important Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import re

In [3]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")  # Punkt Sentence Tokenizer
nltk.download("averaged_perceptron_tagger")  # Part of Speech Tagger
nltk.download("wordnet")  # a lexical database of English; useful for synonyms, hyponyms, etc.

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/marneusz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marneusz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/marneusz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/marneusz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Loading Data

In [5]:
CUR_DATASET = "LIAR"

In [6]:
train_dataset = pd.read_csv(f'./data/{CUR_DATASET}/train.tsv', sep='\t', header = None)
valid_dataset = pd.read_csv(f'./data/{CUR_DATASET}/valid.tsv', sep='\t', header = None)
test_dataset = pd.read_csv(f'./data/{CUR_DATASET}/test.tsv', sep='\t', header = None)
liar_dataset = pd.concat([train_dataset, valid_dataset, test_dataset], axis = 0).reset_index(drop = True)

In [7]:
liar_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [8]:
liar_dataset = liar_dataset.iloc[:, [1, 2]]
liar_dataset = liar_dataset.rename(columns = {1: 'label', 2: 'statements'})
liar_dataset.head()

,label,statements
0,false,Says the Annies List political group supports ...
1,half-true,When did the decline of coal start? It started...
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
3,false,Health care reform legislation is likely to ma...
4,half-true,The economic turnaround started at the end of ...


In [9]:
liar_dataset['label'] = liar_dataset['label'].replace({
    'false' : 0,
    'barely-true' : 0,
    'pants-fire' : 0,
    'half-true' : 1,
    'mostly-true' : 1,
    'true' : 1
})


In [10]:
labels = liar_dataset["label"].values

# Some More EDA

In [11]:
liar_dataset.isnull().sum()

label         0
statements    0
dtype: int64

# Data Preprocessing and Data Preparation

### Removing stopwords

In [12]:
stop_words = stopwords.words('english')

In [13]:
if_stopwords = True

### Preprocessing

In [14]:
preprocessing_text_fn = {
    "no_punctuation": lambda txt: re.sub(r'[^\w\s]','', txt),
    "no_special_symbols": lambda txt: re.sub('[$,#,&]', '', txt),
    "no_digits": lambda txt: re.sub('\d*', '', txt),
    "no_www": lambda txt: re.sub('w{3}', '', txt),
    "no_urls": lambda txt: re.sub('http\S+', '', txt),
    "no_spaces": lambda txt: re.sub('\s+', ' ', txt),
    "no_single_chars": lambda txt: re.sub(r'\s+[a-zA-Z]\s+', '', txt)
}

In [15]:
def preprocess_text(text, pipeline = preprocessing_text_fn):
    text = str(text)
    for fn in pipeline.keys():
        text = pipeline[fn](text)
        
    return text

In [16]:
liar_dataset["statements"] = liar_dataset["statements"].apply(preprocess_text)
liar_dataset.head(10)

,label,statements
0,0,Says the Annies List political group supports ...
1,1,When did the decline of coal start It started ...
2,1,Hillary Clinton agrees with John McCain by vot...
3,0,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...
5,1,The Chicago Bears have had more starting quart...
6,0,Jim Dunnam has not lived in the district he re...
7,1,Im the only person on this stage who has worke...
8,1,However it took million in Oregon Lottery fund...
9,1,Says GOP primary opponents Glenn Grothman and ...


In [17]:
if if_stopwords:
    liar_dataset["statements"] = liar_dataset["statements"].str.lower().str.replace("’", "'")
    liar_dataset["statements"] = liar_dataset["statements"].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


### Lemmatization and Stemming

In [18]:
if_lemmatize = True

In [19]:
if if_lemmatize:
    
    import nltk
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    from nltk.stem import WordNetLemmatizer
    
    wnl = WordNetLemmatizer()
    
    liar_dataset["statements"] = liar_dataset["statements"].str.lower().str.replace("’", "'")
    liar_dataset["statements"] = liar_dataset["statements"].apply(lambda x: ' '.join([wnl.lemmatize(word) for word in word_tokenize(x)]))
    

[nltk_data] Downloading package wordnet to /home/marneusz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/marneusz/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
train_text = liar_dataset["statements"].values

# Model Initialization

In [21]:
from tqdm import tqdm

In [22]:
import torch
if torch.cuda.is_available():    
    device = torch.device('cuda')    
print(device)

cuda


/home/marneusz/anaconda3/envs/nlp-transformers/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [24]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

In [26]:
print(' Original: ', train_text[0])
print('Tokenized: ', tokenizer.tokenize(train_text[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_text[0])))

 Original:  say annies list political group support thirdtrimester abortion demand
Tokenized:  ['say', 'annie', '##s', 'list', 'political', 'group', 'support', 'third', '##tri', '##mes', '##ter', 'abortion', 'demand']
Token IDs:  [2360, 8194, 2015, 2862, 2576, 2177, 2490, 2353, 18886, 7834, 3334, 11324, 5157]


In [27]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Use the 124-layer, 1024-hidden, 16-heads, 340M parameters BERT model with an uncased vocab.
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False, 
)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [28]:
max_len = 0
len_limit = 512
LIMIT = 100_000

indices = []
train_text_filtered = []

for i, text in enumerate(tqdm(train_text)):
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    if len(input_ids) <= LIMIT:
        train_text_filtered.append(text)
        indices.append(i)
print('Max sentence length: ', max_len)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12791/12791 [00:01<00:00, 7124.91it/s]

Max sentence length:  453


In [29]:
labels_filtered = labels[indices]
labels_filtered.shape

(12791,)

In [30]:
# https://www.kaggle.com/code/jeongwonkim10516/nlp-fake-news-with-bert-99-55-top1/notebook

def tokenize_map(sentence, labs='None'):
    
    """A function for tokenize all of the sentences and map the tokens to their word IDs."""
    
    global labels
    
    input_ids = []
    attention_masks = []

    # For every sentence...
    
    for text in tqdm(sentence):
        #   "encode_plus" will:
        
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        
        encoded_dict = tokenizer.encode_plus(
                            text,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            truncation='longest_first', # Activate and control truncation
                            max_length = len_limit,           # Max length according to our text data.
                            padding = 'max_length', # Pad & truncate all sentences.
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the id list. 
        
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        
        attention_masks.append(encoded_dict['attention_mask'])
        
    # Convert the lists into tensors.
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    if labs != 'None': # Setting this for using this definition for both train and test data so labels won't be a problem in our outputs.
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels
    
    return input_ids, attention_masks

In [31]:
train_text_filtered = np.array(train_text)

In [32]:
train_text.shape, train_text_filtered.shape,

((12791,), (12791,))

In [33]:
input_ids, attention_masks, labels_filtered = tokenize_map(train_text_filtered, labels_filtered)
# test_input_ids, test_attention_masks= tokenize_map(test_text)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12791/12791 [00:02<00:00, 4740.82it/s]
/tmp/ipykernel_8413/2816128611.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if labs != 'None': # Setting this for using this definition for both train and test data so labels won't be a problem in our outputs.


## Train and Validation Dataset

In [34]:
import transformers

In [35]:
seed = 10
transformers.set_seed(seed)
torch.manual_seed(seed)

In [36]:
type(attention_masks)

torch.Tensor

In [37]:
labels_filtered.shape, input_ids.shape

(torch.Size([12791]), torch.Size([12791, 512]))

In [38]:
labels_filtered = labels[indices]
labels_filtered.shape

torch.Size([12791])

In [39]:
dataset = TensorDataset(input_ids, attention_masks, labels_filtered)

In [40]:
torch.manual_seed(seed)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_size, val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [41]:
# Train DataLoader
batch_size = 4

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

# Validation DataLoader
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
)

In [42]:
# # Test DataLoader

# test_data = TensorDataset(test_input_ids, test_attention_masks)
# test_sampler = SequentialSampler(test_data)
# test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## Hyperparameters

In [43]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon
            )

In [44]:
num_epochs = 5
total_num_steps = len(train_dataloader) * num_epochs

## Metrics

In [45]:
def accuracy(predictions, labels):
    pred_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return accuracy_score(labels_flat, pred_flat)

def flat_f1_score(predictions, labels):
    pred_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    

    return f1_score(labels_flat, pred_flat, zero_division=0)

## Training

In [46]:
import time

In [47]:
training_stats = []
validations_labels_ep = []
actual_labels_ep = []

total_t0 = time.time()
for i in range(0, num_epochs):
    print('')
    print('Training...')
    print('----- Epoch {:} / {:} -----'.format(i + 1, num_epochs))

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 50 == 0 and not step == 0:
            elapsed = time.time() - t0
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device).to(torch.int64)
        b_input_mask = batch[1].to(device).to(torch.int64)
        b_labels = batch[2].to(device).to(torch.int64)
        
        model.zero_grad()        

        loss = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     labels=b_labels)[0]
        logits = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask,
                       labels=b_labels)[1]

        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        # scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = time.time() - t0

    print('')
    print('  Average training loss: {0:.2f}'.format(avg_train_loss))
    print('  Training epoc h took: {:}'.format(training_time))
    
    print('')
    print('Running Validation...')

    t0 = time.time()
    
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    total_eval_f1 = 0
    nb_eval_steps = 0
    
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            loss = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)[0]

            logits = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)[1]
            
        # Accumulate the validation loss.
        
        total_eval_loss += loss.item()

        # Move logits and labels to CPU:
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and accumulate it over all batches:
        
        total_eval_accuracy += accuracy(logits, label_ids)
        total_eval_f1 += flat_f1_score(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print('  Accuracy: {0:.5f}'.format(avg_val_accuracy))
    
    # Report the final f1 score for this validation run.
    
    avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    print('  F1: {0:.5f}'.format(avg_val_f1))

    # Calculate the average loss over all of the batches.
    
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    
    
    # Measure how long the validation run took:
    
    validation_time = time.time() - t0
    
    print('  Validation Loss: {0:.5f}'.format(avg_val_loss))
    print('  Validation took: {:}'.format(validation_time))

    # Record all statistics from this epoch.
    
    training_stats.append(
        {
            'epoch': i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Val_F1' : avg_val_f1,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )


Training...
----- Epoch 1 / 5 -----
  Batch    50  of  2,558.    Elapsed: 18.022056579589844.
  Batch   100  of  2,558.    Elapsed: 35.16300654411316.
  Batch   150  of  2,558.    Elapsed: 52.399505615234375.
  Batch   200  of  2,558.    Elapsed: 69.4800112247467.
  Batch   250  of  2,558.    Elapsed: 86.67119908332825.
  Batch   300  of  2,558.    Elapsed: 103.80137991905212.
  Batch   350  of  2,558.    Elapsed: 120.94527506828308.
  Batch   400  of  2,558.    Elapsed: 138.27227902412415.
  Batch   450  of  2,558.    Elapsed: 155.8284728527069.
  Batch   500  of  2,558.    Elapsed: 173.03303337097168.
  Batch   550  of  2,558.    Elapsed: 190.4011104106903.
  Batch   600  of  2,558.    Elapsed: 207.76721024513245.
  Batch   650  of  2,558.    Elapsed: 224.97359538078308.
  Batch   700  of  2,558.    Elapsed: 242.29281377792358.
  Batch   750  of  2,558.    Elapsed: 259.6927402019501.
  Batch   800  of  2,558.    Elapsed: 277.0478844642639.
  Batch   850  of  2,558.    Elapsed: 294.5

  Batch 1,700  of  2,558.    Elapsed: 592.9430689811707.
  Batch 1,750  of  2,558.    Elapsed: 610.5048551559448.
  Batch 1,800  of  2,558.    Elapsed: 628.1325466632843.
  Batch 1,850  of  2,558.    Elapsed: 645.6687848567963.
  Batch 1,900  of  2,558.    Elapsed: 663.1005370616913.
  Batch 1,950  of  2,558.    Elapsed: 680.7579939365387.
  Batch 2,000  of  2,558.    Elapsed: 697.9522998332977.
  Batch 2,050  of  2,558.    Elapsed: 715.4444420337677.
  Batch 2,100  of  2,558.    Elapsed: 733.1167652606964.
  Batch 2,150  of  2,558.    Elapsed: 751.2881557941437.
  Batch 2,200  of  2,558.    Elapsed: 768.7774887084961.
  Batch 2,250  of  2,558.    Elapsed: 786.2936744689941.
  Batch 2,300  of  2,558.    Elapsed: 803.9449002742767.
  Batch 2,350  of  2,558.    Elapsed: 821.4651713371277.
  Batch 2,400  of  2,558.    Elapsed: 838.8354873657227.
  Batch 2,450  of  2,558.    Elapsed: 856.1337380409241.
  Batch 2,500  of  2,558.    Elapsed: 873.296046257019.
  Batch 2,550  of  2,558.    Ela

In [48]:
model.save_pretrained("./models/bert_liar_regexp_stopwords_lemmatization")